# Selective Search

Selective Search, first introduced by Uijlings et al. in their 2012 paper,  is a critical piece of computer vision, deep learning, and object detection research.

In their work, Uijlings et al. demonstrated:

1. How images can be **over-segmented to automatically identify locations in an image that could contain an object**

2. That Selective Search is **far more computationally efficient** than exhaustively computing image pyramids and sliding windows (and without loss of accuracy)

3. Selective Search **can be swapped in for any object detection framework** that utilizes image pyramids and sliding windows

Selective Search works by over-segmenting an image using a superpixel algorithm


Selective Search merges superpixels in a hierarchical fashion based on five key similarity measures:

> **1. Color similarity:** 

Computing a 25-bin histogram for each channel of an image, concatenating them together, and obtaining a final descriptor that is 25×3=75-d. Color similarity of any two regions is measured by the histogram intersection distance.

> **2. Texture similarity:** 

Selective Search extracts Gaussian derivatives at 8 orientations per channel (assuming a 3-channel image). These orientations are used to compute a 10-bin histogram per channel, generating a final texture descriptor that is 8x10x=240-d. **To compute texture similarity between any two regions, histogram intersection is once again used.**

> **3. Size similarity:**

The size similarity metric that Selective Search uses **prefers that smaller regions be grouped earlier rather than later.** Anyone who has used Hierarchical Agglomerative Clustering (HAC) algorithms before knows that HACs are prone to clusters reaching a critical mass and then combining everything that they touch. By enforcing smaller regions to merge earlier, we can help prevent a large number of clusters from swallowing up all smaller regions.

> **4. Shape similarity/compatibility:** 

The idea behind shape similarity is that they should be compatible with each other. Two regions are considered “compatible” if they “fit” into each other (thereby filling gaps in our regional proposal generation). Furthermore, shapes that do not touch should not be merged.

> **5. A final meta-similarity measure:** 

A final meta-similarity acts as a linear combination of the color similarity, texture similarity, size similarity, and shape similarity/compatibility.

In [1]:
import cv2
import random
import time

### Read the image and initiate selective search

In [2]:
image = cv2.imread('/media/juan/juan1/pyimage_univ/object_detect_201/opencv-selective-search/dog.jpg')

In [3]:
#Initialize cv2 selective search implementation
#Need to have installed ONLY opencv-contrib-python. Two or more opencv packages together cause conflicts
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [4]:
ss.setBaseImage(image)

Initialization of Selective search requires another step — choosing and setting the internal mode of operation.

- The "fast" method: **switchToSelectiveSearchFast**

- The "quality" method: **switchToSelectiveSearchQuality**

Generally, the faster method will be suitable; however, depending on your application, you might want to sacrifice speed to achieve better quality results

In [5]:
ss.switchToSelectiveSearchFast()
#ss.switchToSelectiveSearchQuality()

### Apply selective search

In [7]:
start = time.time()
rects = ss.process()
end = time.time()
print("Time taken: {:.2f} seconds".format(end-start))
print("length of region proposals by selective search: {}".format(len(rects)))

Time taken: 1.61 seconds
length of region proposals by selective search: 1219


In [11]:
## Visualize  the current subset of region proposals
for i in range(0, len(rects), 100):
    output = image.copy()
    #loop over the subset of region proposals (12 in total)
    for (x, y, w, h) in rects[i:i+100]:
        #draw those regions proposals
        #generate a random color for all the rectangles
        color = [random.randint(0, 255) for j in range(0,3)]
        #get the rectangles
        cv2.rectangle(output, (x,y), (x+w, y+h), color, 2)
    
    cv2.imshow("Out", output)
    cv2.waitKey(0)

cv2.destroyAllWindows()